The data preprocessing phase.
---

After data collection, we cleaned the data since our data set was quite imbalance and had a lot of features. We dropped inull values and rrelevant features such as ID, Case, Number, FBI code, Updated On, ICUR, X and Y Coordinates, Location and Description. There were 34 distinct types of crimes and we dropped those crimes we thought were not very significant. Similar types of crime such as sexual assault, prostitution and sex offenses were mierged together. Hence after preprocessing the primary crime types dropped from 35 to 18. A new csv file was created after pre-processing for analysis.

Download the dataset from link  https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2

and place in dataset folder.



In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/12 16:45:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/12 16:45:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
#spark imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import csv
from pyspark.sql.types import *
from pyspark.sql.functions import format_number, when
import pyspark.sql.functions as F

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [4]:
#change RAM allocation, current = 20GB
#change Core allocation, current = 3 Cores

spark = SparkSession.builder.appName("Project - Chicago crime")\
.config("spark.some.config.option", "some-value")\
.config("spark.driver.memory", "16g")\
.config("spark.driver.cores", '3')\f
.getOrCreate()



In [5]:
crimes_schema = StructType([StructField("ID", IntegerType(), True),
                            StructField("Case Number", StringType(), True),
                            StructField("Date", StringType(), True ),
                            StructField("Block", StringType(), True),
                            StructField("IUCR", StringType(), True),
                            StructField("Primary Type", StringType(), True  ),
                            StructField("Description", StringType(), True ),
                            StructField("Location Description", StringType(), True ),
                            StructField("Arrest", BooleanType(), True),
                            StructField("Domestic", BooleanType(), True),
                            StructField("Beat", IntegerType(), True),
                            StructField("District", IntegerType(), True),
                            StructField("Ward", IntegerType(), True),
                            StructField("Community Area", IntegerType(), True),
                            StructField("FBI Code", StringType(), True ),
                            StructField("X Coordinate", IntegerType(), True),
                            StructField("Y Coordinate", IntegerType(), True ),
                            StructField("Year", IntegerType(), True),
                            StructField("Updated On", StringType(), True ),
                            StructField("Latitude", DoubleType(), True),
                            StructField("Longitude", DoubleType(), True),
                            StructField("Location", StringType(), True )
                            ])

In [6]:
data = spark.read.csv('Dataset/Crimes_-_2001_to_Present.csv',header = True,schema = crimes_schema)

In [7]:
data.count()
print(len(data.columns))

22


In [8]:
data.head(5)

[Row(ID=10224738, Case Number='HY411648', Date='09/05/2015 01:30:00 PM', Block='043XX S WOOD ST', IUCR='0486', Primary Type='BATTERY', Description='DOMESTIC BATTERY SIMPLE', Location Description='RESIDENCE', Arrest=False, Domestic=True, Beat=924, District=9, Ward=12, Community Area=61, FBI Code='08B', X Coordinate=1165074, Y Coordinate=1875917, Year=2015, Updated On='02/10/2018 03:50:01 PM', Latitude=41.815117282, Longitude=-87.669999562, Location='(41.815117282, -87.669999562)'),
 Row(ID=10224739, Case Number='HY411615', Date='09/04/2015 11:30:00 AM', Block='008XX N CENTRAL AVE', IUCR='0870', Primary Type='THEFT', Description='POCKET-PICKING', Location Description='CTA BUS', Arrest=False, Domestic=False, Beat=1511, District=15, Ward=29, Community Area=25, FBI Code='06', X Coordinate=1138875, Y Coordinate=1904869, Year=2015, Updated On='02/10/2018 03:50:01 PM', Latitude=41.895080471, Longitude=-87.765400451, Location='(41.895080471, -87.765400451)'),
 Row(ID=11646166, Case Number='JC21

# Data preprocessing - Spark

**Working on data starting 2010.**

In [9]:

# dataset = data.filter((data['Year'] >2015 & (data['Year'] !=2020))

dataset = data.filter((data['Year'] > 2009))

In [10]:
#dropping columns which non-relative 

dataset = dataset.drop('ID')
dataset = dataset.drop('Case Number')
dataset = dataset.drop('FBI Code')
dataset = dataset.drop('Updated On')
dataset = dataset.drop('IUCR')# Could be useful - four digit codes that law enforcement agencies use to classify criminal incidents when taking individual reports.
dataset = dataset.drop('X Coordinate')
dataset = dataset.drop('Y Coordinate')
dataset = dataset.drop('Location')
dataset = dataset.drop('Description')
dataset = dataset.drop('Domestic')

**Preprocessing Primary types**

In [11]:
primaryCount = dataset.groupby('Primary Type').count()
primaryCount.orderBy('count', ascending=False).show()

+--------------------+------+
|        Primary Type| count|
+--------------------+------+
|               THEFT|817930|
|             BATTERY|666649|
|     CRIMINAL DAMAGE|398114|
|           NARCOTICS|276442|
|             ASSAULT|256070|
|       OTHER OFFENSE|224836|
|  DECEPTIVE PRACTICE|217237|
|            BURGLARY|194661|
| MOTOR VEHICLE THEFT|176903|
|             ROBBERY|141008|
|   CRIMINAL TRESPASS| 87936|
|   WEAPONS VIOLATION| 68915|
|OFFENSE INVOLVING...| 30589|
|PUBLIC PEACE VIOL...| 26815|
|        PROSTITUTION| 15383|
|         SEX OFFENSE| 14537|
| CRIM SEXUAL ASSAULT| 13556|
|INTERFERENCE WITH...| 13395|
|            HOMICIDE|  7751|
|CRIMINAL SEXUAL A...|  6150|
+--------------------+------+
only showing top 20 rows



In [12]:
print('Distinct primary types '+ str(primaryCount.distinct().count()))

Distinct primary types 35


**Drop crimes which are not so significant**

In [13]:
dataset = dataset.filter((dataset['Primary Type']!='STALKING')&
                             (dataset['Primary Type']!='OBSCENITY')&
                             (dataset['Primary Type']!='NON-CRIMINAL (SUBJECT SPECIFIED)')&
                             (dataset['Primary Type']!='GAMBLING')&
                             (dataset['Primary Type']!='NON - CRIMINAL')&
                             (dataset['Primary Type']!='LIQUOR LAW VIOLATION')&
                             (dataset['Primary Type']!='PUBLIC INDECENCY')&
                             (dataset['Primary Type']!='HUMAN TRAFFICKING')&
                             (dataset['Primary Type']!='INTIMIDATION')&
                             (dataset['Primary Type']!='CONCEALED CARRY LICENSE VIOLATION')&
                             (dataset['Primary Type']!='NON-CRIMINAL')&
                             (dataset['Primary Type']!='OTHER NARCOTIC VIOLATION'))

**Merging similar crime types**

In [14]:
dataset = dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'CRIM SEXUAL ASSAULT') | \
                                    (dataset["Primary Type"] == 'PROSTITUTION') ,'SEX OFFENSE').otherwise(dataset['Primary Type']))
dataset=dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'WEAPONS VIOLATION') | \
                                    (dataset["Primary Type"] == 'INTERFERENCE WITH PUBLIC OFFICER') ,'PUBLIC PEACE VIOLATION').otherwise(dataset['Primary Type']))
dataset=dataset.withColumn("Primary Type", \
                               when((dataset["Primary Type"] == 'ARSON') ,'CRIMINAL TRESPASS').otherwise(dataset['Primary Type']))

dataset.show(5)

+--------------------+--------------------+------------+--------------------+------+----+--------+----+--------------+----+------------+-------------+
|                Date|               Block|Primary Type|Location Description|Arrest|Beat|District|Ward|Community Area|Year|    Latitude|    Longitude|
+--------------------+--------------------+------------+--------------------+------+----+--------+----+--------------+----+------------+-------------+
|09/05/2015 01:30:...|     043XX S WOOD ST|     BATTERY|           RESIDENCE| false| 924|       9|  12|            61|2015|41.815117282|-87.669999562|
|09/04/2015 11:30:...| 008XX N CENTRAL AVE|       THEFT|             CTA BUS| false|1511|      15|  29|            25|2015|41.895080471|-87.765400451|
|09/01/2018 12:01:...|082XX S INGLESIDE...|       THEFT|           RESIDENCE| false| 631|       6|   8|            44|2018|        null|         null|
|09/05/2015 12:45:...|   035XX W BARRY AVE|   NARCOTICS|            SIDEWALK|  true|1412|     

In [15]:
primaryCount = dataset.groupby('Primary Type').count()
primaryCount.orderBy('count', ascending=False).show()

+--------------------+------+
|        Primary Type| count|
+--------------------+------+
|               THEFT|817930|
|             BATTERY|666649|
|     CRIMINAL DAMAGE|398114|
|           NARCOTICS|276442|
|             ASSAULT|256070|
|       OTHER OFFENSE|224836|
|  DECEPTIVE PRACTICE|217237|
|            BURGLARY|194661|
| MOTOR VEHICLE THEFT|176903|
|             ROBBERY|141008|
|PUBLIC PEACE VIOL...|109125|
|   CRIMINAL TRESPASS| 93977|
|         SEX OFFENSE| 43476|
|OFFENSE INVOLVING...| 30589|
|            HOMICIDE|  7751|
|CRIMINAL SEXUAL A...|  6150|
|          KIDNAPPING|  2548|
|           RITUALISM|     1|
+--------------------+------+



In [17]:
print('Distinct primary types after preprocessing '+ str(primaryCount.distinct().count()))

Distinct primary types after preprocessing 18


**Extract Day, Month, Week from Date Column**

In [65]:
dataset = dataset.withColumn("Day", F.split(dataset.Date, " ")[0])
dataset = dataset.withColumn("Day", F.to_date(dataset.Day, "MM/dd/yyyy"))
dataset = dataset.withColumn("Month", F.month(dataset.Day))
dataset = dataset.withColumn("WeekDay", F.dayofweek(dataset.Day))
dataset = dataset.withColumn("Year", F.year(dataset.Day))
dataset = dataset.na.drop()

#dataset = dataset.drop('Day')
#dataset = dataset.drop('Date')
dataset.head(5)

[Row(Date='03/17/2020 09:30:00 PM', Block='039XX N LECLAIRE AVE', Primary Type='THEFT', Location Description='STREET', Arrest=False, Beat=1634, District=16, Ward=45, Community Area=15, Year=2020, Latitude=41.952051946, Longitude=-87.754660372, Day=datetime.date(2020, 3, 17), Month=3, WeekDay=3),
 Row(Date='09/24/2019 08:00:00 AM', Block='022XX S MICHIGAN AVE', Primary Type='DECEPTIVE PRACTICE', Location Description='COMMERCIAL / BUSINESS OFFICE', Arrest=False, Beat=132, District=1, Ward=3, Community Area=33, Year=2019, Latitude=41.852248185, Longitude=-87.623786256, Day=datetime.date(2019, 9, 24), Month=9, WeekDay=3),
 Row(Date='10/13/2019 08:30:00 PM', Block='024XX W CHICAGO AVE', Primary Type='THEFT', Location Description='GROCERY FOOD STORE', Arrest=False, Beat=1221, District=12, Ward=26, Community Area=24, Year=2019, Latitude=41.895732399, Longitude=-87.687784384, Day=datetime.date(2019, 10, 13), Month=10, WeekDay=1),
 Row(Date='03/18/2020 02:03:00 AM', Block='039XX W JACKSON BLVD'

Ignore Latitude and Longitude outside Chichago

In [66]:
dataset=dataset\
        .filter((dataset["Latitude"] < 45)
             & (dataset["Latitude"] > 40)
             & (dataset["Longitude"] < -85)
             & (dataset["Longitude"] > -90))

**Dropping Null values in the data.**

In [67]:
dataset.head(5)

[Row(Date='03/17/2020 09:30:00 PM', Block='039XX N LECLAIRE AVE', Primary Type='THEFT', Location Description='STREET', Arrest=False, Beat=1634, District=16, Ward=45, Community Area=15, Year=2020, Latitude=41.952051946, Longitude=-87.754660372, Day=datetime.date(2020, 3, 17), Month=3, WeekDay=3),
 Row(Date='09/24/2019 08:00:00 AM', Block='022XX S MICHIGAN AVE', Primary Type='DECEPTIVE PRACTICE', Location Description='COMMERCIAL / BUSINESS OFFICE', Arrest=False, Beat=132, District=1, Ward=3, Community Area=33, Year=2019, Latitude=41.852248185, Longitude=-87.623786256, Day=datetime.date(2019, 9, 24), Month=9, WeekDay=3),
 Row(Date='10/13/2019 08:30:00 PM', Block='024XX W CHICAGO AVE', Primary Type='THEFT', Location Description='GROCERY FOOD STORE', Arrest=False, Beat=1221, District=12, Ward=26, Community Area=24, Year=2019, Latitude=41.895732399, Longitude=-87.687784384, Day=datetime.date(2019, 10, 13), Month=10, WeekDay=1),
 Row(Date='03/18/2020 02:03:00 AM', Block='039XX W JACKSON BLVD'

In [68]:
dataset.count()

942357

In [69]:
dataset=dataset.na.drop()

In [70]:
dataset.repartition(1).write.csv('pre_processed_data',mode="append",header=True)